In [1]:
from pyspark.sql import SparkSession, functions as F, types as T

In [2]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

24/05/24 13:59:11 WARN Utils: Your hostname, MacBook-Pro-Almin.local resolves to a loopback address: 127.0.0.1; using 192.168.68.103 instead (on interface en0)
24/05/24 13:59:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/24 13:59:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(spark.version)

3.4.1


In [4]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [17]:
films_per_category = film_category_df.groupBy("category_id").count()
films_per_category.orderBy(F.col("count").desc()).show()

+-----------+-----+
|category_id|count|
+-----------+-----+
|         15|   74|
|          9|   73|
|          8|   69|
|          6|   68|
|          2|   66|
|          1|   64|
|         13|   63|
|          7|   62|
|         10|   61|
|         14|   61|
|          3|   60|
|          5|   58|
|         16|   57|
|          4|   57|
|         11|   56|
|         12|   51|
+-----------+-----+


2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [27]:
films_by_rental = (rental_df
                   .join(inventory_df, on="inventory_id")
                   .groupBy("film_id").count())
popular_actors = (films_by_rental
                  .join(film_actor_df, on="film_id")
                  .join(actor_df, on="actor_id"))

popular_actors.orderBy(F.col("count").desc()).select("first_name", "last_name", "count").show(10)

+----------+---------+-----+
|first_name|last_name|count|
+----------+---------+-----+
|       RIP| CRAWFORD|   34|
|       TIM|  HACKMAN|   34|
|      BURT|   TEMPLE|   34|
|   KIRSTEN|   AKROYD|   34|
|  CHARLIZE|    DENCH|   34|
|      GARY|  PHOENIX|   34|
|    WARREN|  JACKMAN|   33|
|       TOM|  MIRANDA|   33|
|     CHRIS|  BRIDGES|   33|
|      JUDY|     DEAN|   33|
+----------+---------+-----+


3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [38]:
rental_categories_with_money = (rental_df
                                .join(inventory_df, on="inventory_id")
                                .join(film_category_df, on="film_id")
                                .join(payment_df, on="rental_id")
                                .join(category_df, on="category_id")
                                .select("name", "amount").groupBy("name").sum())

rental_categories_with_money.orderBy(F.col("sum(amount)").desc()).show(1)

+------+-----------------+
|  name|      sum(amount)|
+------+-----------------+
|Sports|5314.209999999848|
+------+-----------------+


4.
Вивести назви фільмів, яких не має в inventory.

In [51]:
non_inventory_films = film_df.join(inventory_df, on="film_id", how="anti")
non_inventory_films.select("title").show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+


5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [68]:
actors_child = (actor_df
                      .join(film_actor_df, on="actor_id")
                      .join(film_category_df, on="film_id")
                      .join(category_df, on="category_id")
                      .filter(F.col("name") == "Children")
                      .withColumn("full_name", F.concat(F.col("first_name"), F.lit(" "), F.col("last_name")))
                      .groupBy("full_name").count()
                )

actors_child.orderBy(F.col("count").desc()).select("full_name").show(3)

+------------+
|   full_name|
+------------+
|HELEN VOIGHT|
| SUSAN DAVIS|
|  MARY TANDY|
+------------+


Stop Spark session:

In [69]:
spark.stop()